In [1]:
import heist
import helpers
import torch.distributions
import torch

import gym
import random
import numpy as np
from helpers import generate_action, load_model
from procgen import ProcgenGym3Env
import imageio
import matplotlib.pyplot as plt
import typing
import math

from procgen import ProcgenGym3Env
import struct
import typing
from typing import Tuple, Dict, Callable, List, Optional
from dataclasses import dataclass
from src.policies_impala import ImpalaCNN
from procgen_tools.procgen_wrappers import VecExtractDictObs, TransposeFrame, ScaledFloatFrame

from gym3 import ToBaselinesVecEnv
import seaborn as sns


%load_ext autoreload
%autoreload 2

building procgen...

/Users/bensturgeon/mambaforge/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(
/Users/bensturgeon/mambaforge/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


done


In [6]:
import torch
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import src.probing as probing

categories  = {
        "gem": [],
        "blue_key": [],
        "green_key": [],
        "red_key": [],
        "blue_lock": [],
        "green_lock": [],
        "red_lock": []
    }

# Create classified dataset
classified_dataset = heist.create_classified_dataset(num_samples_per_category=900, num_levels=0)

# Load the model
model = helpers.load_interpretable_model(model_path="../model_interpretable.pt")

layer_paths = helpers.get_model_layer_names(model)
# Create activation dataset
activation_dataset = helpers.create_activation_dataset(classified_dataset, model, layer_paths, categories)



In [8]:
layer_paths= {
    1: 'conv1a',
    2: 'pool1',
    3: 'conv2a',
    4: 'conv2b',
    5: 'pool2',
    6: 'conv3a',
    7: 'pool3',
    8: 'conv4a',
    9: 'pool4',
    10: 'fc1',
    11: 'fc2',
    12: 'fc3',
    13: 'value_fc',
    14: 'dropout_conv',
    15: 'dropout_fc'
}


KeyError: 'x_test'

In [11]:
results_dataframe, probes = probing.linear_probe_per_category_using_probes(activation_dataset, 'fc1', model_type='classifier', test_size=0.2, random_state=42)
print(results_dataframe["test_accuracy"])
# results_dataframe = probing.linear_probe_per_category_using_probes(activation_dataset, 'fc2', model_type='classifier', test_size=0.2, random_state=42)
# print(results_dataframe["test_accuracy"])
# results_dataframe = probing.linear_probe_per_category_using_probes(activation_dataset, 'fc3', model_type='classifier', test_size=0.2, random_state=42)
# print(results_dataframe["test_accuracy"])


/Users/bensturgeon/Library/Caches/pypoetry/virtualenvs/ai-safety-camp-Y8XZewIj-py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bensturgeon/Library/Caches/pypoetry/virtualenvs/ai-safety-camp-Y8XZewIj-py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer 

0    0.962698
1    0.975397
2    0.865873
3    0.880159
4    0.842857
5    0.846825
6    0.921429
Name: test_accuracy, dtype: float64


/Users/bensturgeon/Library/Caches/pypoetry/virtualenvs/ai-safety-camp-Y8XZewIj-py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bensturgeon/Library/Caches/pypoetry/virtualenvs/ai-safety-camp-Y8XZewIj-py3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer 

In [12]:
import matplotlib.pyplot as plt

new_classification_data = results_dataframe["classification_report"]

for key,val in new_classification_data.items():
    print(val["0"]["f1-score"])

0.9779653070792311
0.9855746859004187
0.9243170622480967
0.9303826648224989
0.9121561668145519
0.9147149801148917
0.9534555712270804


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

def analyze_probe_weights(weights, objectives):
    """Analyze and visualize the weights of a linear probe."""
    
    # 1. Magnitude Analysis
    importance = np.abs(weights)
    top_neurons = np.argsort(importance, axis=1)[:, ::-1][:, :10]  # Top 10 neurons for each objective
    
    # 2. Visualization
    plt.figure(figsize=(15, 10))
    plt.imshow(weights, cmap='coolwarm', aspect='auto')
    plt.colorbar()
    plt.title('Probe Weights for Each Objective')
    plt.xlabel('Dense Layer Neurons')
    plt.ylabel('Objectives')
    plt.yticks(range(len(objectives)), objectives)
    plt.show()
    
    # 3. Per-Objective Analysis
    for i, objective in enumerate(objectives):
        plt.figure(figsize=(10, 5))
        plt.bar(range(len(weights[i])), weights[i])
        plt.title(f'Weight Distribution for {objective}')
        plt.xlabel('Neuron Index')
        plt.ylabel('Weight')
        plt.show()
    
    # 4. Shared vs. Unique Neurons
    threshold = np.percentile(np.abs(weights), 95)  # Top 5% of weights
    important_neurons = np.abs(weights) > threshold
    shared_neurons = np.sum(important_neurons, axis=0) > 1
    unique_neurons = np.sum(important_neurons, axis=0) == 1
    
    print(f"Number of shared important neurons: {np.sum(shared_neurons)}")
    print(f"Number of unique important neurons: {np.sum(unique_neurons)}")
    
    # 5. Correlation Analysis
    corr_matrix = np.corrcoef(weights)
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', xticklabels=objectives, yticklabels=objectives)
    plt.title('Correlation between Objective Weight Vectors')
    plt.show()
    
    # 6. Dimensionality Reduction
    pca = PCA(n_components=2)
    weights_2d = pca.fit_transform(weights)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(weights_2d[:, 0], weights_2d[:, 1])
    for i, obj in enumerate(objectives):
        plt.annotate(obj, (weights_2d[i, 0], weights_2d[i, 1]))
    plt.title('PCA of Weight Vectors')
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.show()
    
    return top_neurons, shared_neurons, unique_neurons

# Usage example
# Assuming you have trained a linear probe and have the weights and objectives

# Example data (replace with your actual data)
weights = probes # 7 objectives, 100 neurons
objectives = ['gem', 'blue_key', 'green_key', 'red_key', 'blue_lock', 'green_lock', 'red_lock']

top_neurons, shared_neurons, unique_neurons = analyze_probe_weights(weights, objectives)

# print("Top 5 neurons for each objective:")
# for i, obj in enumerate(objectives):
#     print(f"{obj}: {top_neurons[i][:5]}")

# print("\nIndices of shared important neurons:", np.where(shared_neurons)[0])
# print("Indices of unique important neurons:", np.where(unique_neurons)[0])

# # 7. Activation Pattern Analysis (you would need to implement this with your actual data)
# def analyze_activation_patterns(activations, top_neurons, objectives):
#     """Analyze activation patterns of top neurons for each objective."""
#     for i, obj in enumerate(objectives):
#         plt.figure(figsize=(12, 5))
#         for j, neuron in enumerate(top_neurons[i][:5]):
#             plt.plot(activations[:, neuron], label=f'Neuron {neuron}')
#         plt.title(f'Activation Patterns of Top 5 Neurons for {obj}')
#         plt.xlabel('Sample')
#         plt.ylabel('Activation')
#         plt.legend()
#         plt.show()

# # Usage (you would need your actual activation data)
# # activations = ... # Shape: (n_samples, n_neurons)
# # analyze_activation_patterns(activations, top_neurons, objectives)

# # 8. Ablation Studies (you would need to implement this with your actual model and data)
# def ablation_study(model, x_test, y_test, top_neurons, objectives):
#     """Perform ablation study by zeroing out top neurons for each objective."""
#     base_score = model.score(x_test, y_test)
#     for i, obj in enumerate(objectives):
#         x_test_ablated = x_test.copy()
#         x_test_ablated[:, top_neurons[i][:10]] = 0  # Zero out top 10 neurons
#         ablated_score = model.score(x_test_ablated, y_test)
#         print(f"Ablation result for {obj}:")
#         print(f"  Base score: {base_score:.4f}")
#         print(f"  Ablated score: {ablated_score:.4f}")
#         print(f"  Performance drop: {base_score - ablated_score:.4f}")

# # Usage (you would need your actual model and test data)
# # ablation_study(model, x_test, y_test, top_neurons, objectives)

TypeError: bad operand type for abs(): 'LogisticRegression'